In [1]:
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import requests
import json
import prettytable
import geopandas as gpd
import fiona

In [68]:
msaData = pd.read_excel('../data.xlsx', sheet_name="msaData")
msaData = msaData.set_index(['msa'])
msaData = msaData.loc[msaData.index.isin(['Albany-Lebanon, OR','Anniston-Oxford, AL','Austin-Round Rock-Georgetown, TX'])]
msaData.reset_index(inplace=True)

In [77]:
msaShape = gpd.read_file('../../../data/spatial/msa/usMsaData.geojson')

In [70]:
# msaShape = msaShape[['cbsa','geoid','msa','STATE','December_2020_LaborForce','December_2020_URate',
#              'NUMBER_AIRPORTS','AIRPORT','CODE','NS_DOMESTIC','NS_INTERNATIONAL','NS_TOTAL','geometry']]

In [71]:
msaData = pd.merge(msaData, msaShape, how='left', on=['msa','STATE'])

In [73]:
msaData.shape

(3, 77)

In [78]:
msaShape.shape

(381, 77)

In [75]:
msaData = msaData[['cbsa', 'geoid', 'msa', 'STATE', 'December_2020_LaborForce',
       'December_2020_URate', 'NUMBER_AIRPORTS', 'AIRPORT', 'CODE',
       'NS_DOMESTIC', 'NS_INTERNATIONAL', 'NS_TOTAL', 'Avg. Local Sales Tax',
       'State Sales Tax', 'Total Sales Tax', 'Property Tax per Capita',
       'Property Tax Rank', 'Property Tax Throw Back', 'Taxable Wages Rate',
       'New Employers Rate', 'Taxable Wage Base', 'Payroll Tax Notes',
       'Individual Tax Rate', 'Individual Tax Brackets',
       'Graduated Individual Tax', 'Franchise Tax Rate', 'Corporate Tax Rate',
       'Corporate Tax Brackets', 'Graduated Corporate Tax', 'STATE_CODB',
       'Fortune 500', 'Fortune 1000', 'HOUSING_PERMITS1', 'HOUSING_PERMITS2',
       'PERMIT_CHANGE', 'SCORE', 'TOTPOP_CY', 'TOTPOP_FY', 'POP_ANNUAL_GROWTH',
       'MEDAGE_CY', 'MEDHHINC_CY', 'HOME_VALUE', 'COLLEGE_DEGREE', 'DIV_INDEX',
       'URBAN_AREA', 'COMP_INDEX', 'GROCERY', 'HOUSING', 'UTILITIES',
       'TRANSPORTATION', 'HEALTH_CARE', 'MISC', 'APT_RENT',
       'PHYSICAL_SCIENCES_ENROLLMENT', 'BIO_LIFE_ENROLLMENT',
       'BUSINESS_MGMT_ENROLLMENT', 'EDUCATION_ENROLLMENT',
       'ENGINEERING_ENROLLMENT', 'LAW_ENROLLMENT', 'MEDICINE_ENROLLMENT',
       'ENROLLMENT', 'COMPLETIONS', 'NUMBER_OF_COLLEGES', 'MSA_CODB',
       'UNIT_LABOR_COST', 'ENERGY_COST', 'STATE_LOCAL_TAX', 'OFFICE_RENT',
       'JAN_RAIN', 'JAN_SNOW', 'JAN_TEMP_MAX', 'JAN_TEMP_MIN', 'JULY_RAIN',
       'JULY_SNOW', 'JULY_TEMP_MAX', 'JULY_TEMP_MIN', 'geometry']]

In [76]:
msaData.head()

,cbsa,geoid,msa,STATE,December_2020_LaborForce,December_2020_URate,NUMBER_AIRPORTS,AIRPORT,CODE,NS_DOMESTIC,NS_INTERNATIONAL,NS_TOTAL,Avg. Local Sales Tax,State Sales Tax,Total Sales Tax,Property Tax per Capita,Property Tax Rank,Property Tax Throw Back,Taxable Wages Rate,New Employers Rate,Taxable Wage Base,Payroll Tax Notes,Individual Tax Rate,Individual Tax Brackets,Graduated Individual Tax,Franchise Tax Rate,Corporate Tax Rate,Corporate Tax Brackets,Graduated Corporate Tax,STATE_CODB,Fortune 500,Fortune 1000,HOUSING_PERMITS1,HOUSING_PERMITS2,PERMIT_CHANGE,SCORE,TOTPOP_CY,TOTPOP_FY,POP_ANNUAL_GROWTH,MEDAGE_CY,MEDHHINC_CY,HOME_VALUE,COLLEGE_DEGREE,DIV_INDEX,URBAN_AREA,COMP_INDEX,GROCERY,HOUSING,UTILITIES,TRANSPORTATION,HEALTH_CARE,MISC,APT_RENT,PHYSICAL_SCIENCES_ENROLLMENT,BIO_LIFE_ENROLLMENT,BUSINESS_MGMT_ENROLLMENT,EDUCATION_ENROLLMENT,ENGINEERING_ENROLLMENT,LAW_ENROLLMENT,MEDICINE_ENROLLMENT,ENROLLMENT,COMPLETIONS,NUMBER_OF_COLLEGES,MSA_CODB,UNIT_LABOR_COST,ENERGY_COST,STATE_LOCAL_TAX,OFFICE_RENT,JAN_RAIN,JAN_SNOW,JAN_TEMP_MAX,JAN_TEMP_MIN,JULY_RAIN,JULY_SNOW,JULY_TEMP_MAX,JULY_TEMP_MIN,geometry
0,10540,4110540,"Albany-Lebanon, OR",OR,60096.0,6.0,0,None,None,0,0,0,0.00,0.00,0.00,1487.0,27.0,Yes,0.70 - 5.40%,2.1,42100,Includes special payroll tax offset: 0.09%,9.9,4,Yes,No entity-level tax measured by net worth/capi...,7.6,2,Yes,102,0,0,446.0,384.0,-0.139,NaN,131026.0,138123.0,1.06,41.0,55420.0,269787.0,0.209,35.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,91,103,96,92,58,9.7,11.2,42.0,30.0,0.9,0.0,81.0,49.0,POINT (-18.17322 8.41703)
1,11500,0111500,"Anniston-Oxford, AL",AL,44289.0,4.4,0,None,None,0,0,0,5.22,4.00,9.22,582.0,50.0,Yes,0.65 - 6.80%,2.7,8000,Includes 0.06% employment security assessment,5.0,3,Yes,"""Business privilege tax imposed at following g...",6.5,1,No,100,0,0,66.0,72.0,0.091,NaN,116592.0,115051.0,-0.27,40.0,48593.0,124618.0,0.187,46.5,Anniston-Calhoun County AL,83.4,87.4,59.4,126.6,87.5,81.5,88.5,669.0,70.0,446.0,991.0,1025.0,0.0,0.0,0.0,NaN,NaN,NaN,96,109,106,67,62,4.5,0.7,54.0,33.0,4.6,0.0,90.0,70.0,POINT (8.84361 -4.52126)
2,12420,4812420,"Austin-Round Rock-Georgetown, TX",TX,1267150.0,5.1,0,Austin,AUS,55,8,63,1.94,6.25,8.19,1872.0,13.0,NaN,0.31 - 6.31%,2.7,9000,NaN,0.0,1,No,"Revised franchise tax, otherwise known as marg...",0.0,1,No,92,1,2,17030.0,18145.0,0.065,100.0,2296794.0,2610839.0,2.60,34.0,78659.0,293890.0,0.455,73.6,Austin TX,101.8,92.2,109.8,95.0,89.6,108.6,103.2,1530.0,2553.0,7487.0,13958.0,3174.0,8485.0,1069.0,0.0,176444.0,30450.0,25.0,100,91,99,80,126,2.3,0.2,61.0,38.0,2.1,0.0,95.0,72.0,POINT (-0.96286 -8.45697)


In [91]:
data = pd.concat( [msaShape, msaData] ).reset_index( drop=True )

In [92]:
data.drop_duplicates(subset=['cbsa','geoid','msa','STATE'], keep='last', inplace=True)
data.sort_values( 'msa', inplace=True)

In [144]:
data = gpd.read_file('../../../data/spatial/msa/usMsaData.geojson')

In [145]:
bls = pd.read_excel( '../msaBlsSum.xlsx', usecols="A,E,F", converters={'cbsa':str,'geoid':str})


In [146]:
bls.shape

(381, 3)

In [147]:
data.shape

(381, 77)

In [148]:
data = data[['cbsa', 'geoid', 'msa', 'STATE','NUMBER_AIRPORTS', 'AIRPORT', 'CODE',
       'NS_DOMESTIC', 'NS_INTERNATIONAL', 'NS_TOTAL', 'Avg. Local Sales Tax',
       'State Sales Tax', 'Total Sales Tax', 'Property Tax per Capita',
       'Property Tax Rank', 'Property Tax Throw Back', 'Taxable Wages Rate',
       'New Employers Rate', 'Taxable Wage Base', 'Payroll Tax Notes',
       'Individual Tax Rate', 'Individual Tax Brackets',
       'Graduated Individual Tax', 'Franchise Tax Rate', 'Corporate Tax Rate',
       'Corporate Tax Brackets', 'Graduated Corporate Tax', 'STATE_CODB',
       'Fortune 500', 'Fortune 1000', 'HOUSING_PERMITS1', 'HOUSING_PERMITS2',
       'PERMIT_CHANGE', 'SCORE', 'TOTPOP_CY', 'TOTPOP_FY', 'POP_ANNUAL_GROWTH',
       'MEDAGE_CY', 'MEDHHINC_CY', 'HOME_VALUE', 'COLLEGE_DEGREE', 'DIV_INDEX',
       'URBAN_AREA', 'COMP_INDEX', 'GROCERY', 'HOUSING', 'UTILITIES',
       'TRANSPORTATION', 'HEALTH_CARE', 'MISC', 'APT_RENT',
       'PHYSICAL_SCIENCES_ENROLLMENT', 'BIO_LIFE_ENROLLMENT',
       'BUSINESS_MGMT_ENROLLMENT', 'EDUCATION_ENROLLMENT',
       'ENGINEERING_ENROLLMENT', 'LAW_ENROLLMENT', 'MEDICINE_ENROLLMENT',
       'ENROLLMENT', 'COMPLETIONS', 'NUMBER_OF_COLLEGES', 'MSA_CODB',
       'UNIT_LABOR_COST', 'ENERGY_COST', 'STATE_LOCAL_TAX', 'OFFICE_RENT',
       'JAN_RAIN', 'JAN_SNOW', 'JAN_TEMP_MAX', 'JAN_TEMP_MIN', 'JULY_RAIN',
       'JULY_SNOW', 'JULY_TEMP_MAX', 'JULY_TEMP_MIN', 'geometry']]

In [149]:
data.columns

Index(['cbsa', 'geoid', 'msa', 'STATE', 'NUMBER_AIRPORTS', 'AIRPORT', 'CODE',
       'NS_DOMESTIC', 'NS_INTERNATIONAL', 'NS_TOTAL', 'Avg. Local Sales Tax',
       'State Sales Tax', 'Total Sales Tax', 'Property Tax per Capita',
       'Property Tax Rank', 'Property Tax Throw Back', 'Taxable Wages Rate',
       'New Employers Rate', 'Taxable Wage Base', 'Payroll Tax Notes',
       'Individual Tax Rate', 'Individual Tax Brackets',
       'Graduated Individual Tax', 'Franchise Tax Rate', 'Corporate Tax Rate',
       'Corporate Tax Brackets', 'Graduated Corporate Tax', 'STATE_CODB',
       'Fortune 500', 'Fortune 1000', 'HOUSING_PERMITS1', 'HOUSING_PERMITS2',
       'PERMIT_CHANGE', 'SCORE', 'TOTPOP_CY', 'TOTPOP_FY', 'POP_ANNUAL_GROWTH',
       'MEDAGE_CY', 'MEDHHINC_CY', 'HOME_VALUE', 'COLLEGE_DEGREE', 'DIV_INDEX',
       'URBAN_AREA', 'COMP_INDEX', 'GROCERY', 'HOUSING', 'UTILITIES',
       'TRANSPORTATION', 'HEALTH_CARE', 'MISC', 'APT_RENT',
       'PHYSICAL_SCIENCES_ENROLLMENT', 'BIO_LI

In [150]:
data = pd.merge(data, bls, how='left', on='cbsa')

In [151]:
data = data[['cbsa', 'geoid', 'msa', 'STATE', 'January_2021_LaborForce',
       'January_2021_URate', 'NUMBER_AIRPORTS', 'AIRPORT', 'CODE',
       'NS_DOMESTIC', 'NS_INTERNATIONAL', 'NS_TOTAL', 'Avg. Local Sales Tax',
       'State Sales Tax', 'Total Sales Tax', 'Property Tax per Capita',
       'Property Tax Rank', 'Property Tax Throw Back', 'Taxable Wages Rate',
       'New Employers Rate', 'Taxable Wage Base', 'Payroll Tax Notes',
       'Individual Tax Rate', 'Individual Tax Brackets',
       'Graduated Individual Tax', 'Franchise Tax Rate', 'Corporate Tax Rate',
       'Corporate Tax Brackets', 'Graduated Corporate Tax', 'STATE_CODB',
       'Fortune 500', 'Fortune 1000', 'HOUSING_PERMITS1', 'HOUSING_PERMITS2',
       'PERMIT_CHANGE', 'SCORE', 'TOTPOP_CY', 'TOTPOP_FY', 'POP_ANNUAL_GROWTH',
       'MEDAGE_CY', 'MEDHHINC_CY', 'HOME_VALUE', 'COLLEGE_DEGREE', 'DIV_INDEX',
       'URBAN_AREA', 'COMP_INDEX', 'GROCERY', 'HOUSING', 'UTILITIES',
       'TRANSPORTATION', 'HEALTH_CARE', 'MISC', 'APT_RENT',
       'PHYSICAL_SCIENCES_ENROLLMENT', 'BIO_LIFE_ENROLLMENT',
       'BUSINESS_MGMT_ENROLLMENT', 'EDUCATION_ENROLLMENT',
       'ENGINEERING_ENROLLMENT', 'LAW_ENROLLMENT', 'MEDICINE_ENROLLMENT',
       'ENROLLMENT', 'COMPLETIONS', 'NUMBER_OF_COLLEGES', 'MSA_CODB',
       'UNIT_LABOR_COST', 'ENERGY_COST', 'STATE_LOCAL_TAX', 'OFFICE_RENT',
       'JAN_RAIN', 'JAN_SNOW', 'JAN_TEMP_MAX', 'JAN_TEMP_MIN', 'JULY_RAIN',
       'JULY_SNOW', 'JULY_TEMP_MAX', 'JULY_TEMP_MIN', 'geometry']]

In [152]:
crs = {'init': 'epsg:4326'}
data = gpd.GeoDataFrame(data, crs=crs, geometry=data['geometry'])

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [143]:
data.to_csv('deleteMe.csv', index=False)

In [158]:
data.to_file('../../../data/spatial/msa/usMsaData.geojson', driver="GeoJSON")

In [175]:
# get the right property tax throwback values - "NA" is acceptable; cannot read unknown
app = pd.read_excel('../data.xlsx', sheet_name="corporateApportionment")
# get the column for corporate income apportionment - "single factor sales"
data = gpd.read_file('../../../data/spatial/msa/usMsaData.geojson')

In [166]:
data.shape

(381, 78)

In [165]:
data = pd.merge( data, app, how='left', on='STATE')

In [167]:
data = data[['cbsa', 'geoid', 'msa', 'STATE', 'January_2021_LaborForce',
       'January_2021_URate', 'NUMBER_AIRPORTS', 'AIRPORT', 'CODE',
       'NS_DOMESTIC', 'NS_INTERNATIONAL', 'NS_TOTAL', 'Avg. Local Sales Tax',
       'State Sales Tax', 'Total Sales Tax', 'Property Tax per Capita',
       'Property Tax Rank', 'Property Tax Throw Back', 'Taxable Wages Rate',
       'New Employers Rate', 'Taxable Wage Base', 'Payroll Tax Notes',
       'Individual Tax Rate', 'Individual Tax Brackets',
       'Graduated Individual Tax', 'Franchise Tax Rate', 'Corporate Tax Rate','APPORTIONMENT',
       'Corporate Tax Brackets', 'Graduated Corporate Tax', 'STATE_CODB',
       'Fortune 500', 'Fortune 1000', 'HOUSING_PERMITS1', 'HOUSING_PERMITS2',
       'PERMIT_CHANGE', 'SCORE', 'TOTPOP_CY', 'TOTPOP_FY', 'POP_ANNUAL_GROWTH',
       'MEDAGE_CY', 'MEDHHINC_CY', 'HOME_VALUE', 'COLLEGE_DEGREE', 'DIV_INDEX',
       'URBAN_AREA', 'COMP_INDEX', 'GROCERY', 'HOUSING', 'UTILITIES',
       'TRANSPORTATION', 'HEALTH_CARE', 'MISC', 'APT_RENT',
       'PHYSICAL_SCIENCES_ENROLLMENT', 'BIO_LIFE_ENROLLMENT',
       'BUSINESS_MGMT_ENROLLMENT', 'EDUCATION_ENROLLMENT',
       'ENGINEERING_ENROLLMENT', 'LAW_ENROLLMENT', 'MEDICINE_ENROLLMENT',
       'ENROLLMENT', 'COMPLETIONS', 'NUMBER_OF_COLLEGES', 'MSA_CODB',
       'UNIT_LABOR_COST', 'ENERGY_COST', 'STATE_LOCAL_TAX', 'OFFICE_RENT',
       'JAN_RAIN', 'JAN_SNOW', 'JAN_TEMP_MAX', 'JAN_TEMP_MIN', 'JULY_RAIN',
       'JULY_SNOW', 'JULY_TEMP_MAX', 'JULY_TEMP_MIN', 'geometry']]

In [168]:
crs = {'init': 'epsg:4326'}
data = gpd.GeoDataFrame(data, crs=crs, geometry=data['geometry'])

/Users/mmainza/opt/anaconda3/lib/python3.7/site-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [176]:
data['Property Tax Throw Back'].fillna('N/A', inplace=True)

In [179]:
data.to_file('../../../data/spatial/msa/usMsaData.geojson', driver="GeoJSON")

In [177]:
data.tail(20)

,cbsa,geoid,msa,STATE,January_2021_LaborForce,January_2021_URate,NUMBER_AIRPORTS,AIRPORT,CODE,NS_DOMESTIC,NS_INTERNATIONAL,NS_TOTAL,Avg. Local Sales Tax,State Sales Tax,Total Sales Tax,Property Tax per Capita,Property Tax Rank,Property Tax Throw Back,Taxable Wages Rate,New Employers Rate,Taxable Wage Base,Payroll Tax Notes,Individual Tax Rate,Individual Tax Brackets,Graduated Individual Tax,Franchise Tax Rate,Corporate Tax Rate,APPORTIONMENT,Corporate Tax Brackets,Graduated Corporate Tax,STATE_CODB,Fortune 500,Fortune 1000,HOUSING_PERMITS1,HOUSING_PERMITS2,PERMIT_CHANGE,SCORE,TOTPOP_CY,TOTPOP_FY,POP_ANNUAL_GROWTH,MEDAGE_CY,MEDHHINC_CY,HOME_VALUE,COLLEGE_DEGREE,DIV_INDEX,URBAN_AREA,COMP_INDEX,GROCERY,HOUSING,UTILITIES,TRANSPORTATION,HEALTH_CARE,MISC,APT_RENT,PHYSICAL_SCIENCES_ENROLLMENT,BIO_LIFE_ENROLLMENT,BUSINESS_MGMT_ENROLLMENT,EDUCATION_ENROLLMENT,ENGINEERING_ENROLLMENT,LAW_ENROLLMENT,MEDICINE_ENROLLMENT,ENROLLMENT,COMPLETIONS,NUMBER_OF_COLLEGES,MSA_CODB,UNIT_LABOR_COST,ENERGY_COST,STATE_LOCAL_TAX,OFFICE_RENT,JAN_RAIN,JAN_SNOW,JAN_TEMP_MAX,JAN_TEMP_MIN,JULY_RAIN,JULY_SNOW,JULY_TEMP_MAX,JULY_TEMP_MIN,geometry
361,47580,1347580,"Warner Robins, GA",GA,87490,4.5,0,None,None,0,0,0,3.31,4.00,7.31,1161.0,33.0,No,0.04 - 7.56%,2.70,9500.0,Includes 0.06% administrative assessment excep...,5.75,6.0,Yes,Graduated tax from minimum of $125 to maximum ...,5.75,Single factor sales,1.0,No,94.0,0.0,0.0,895.0,996.0,0.113,NaN,188183.0,199908.0,1.22,36.0,56705.0,167359.0,0.280,62.6,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.0,240.0,263.0,305.0,0.0,0.0,0.0,15206.0,1186.0,2.0,82.0,89.0,87.0,80.0,59.0,4.6,0.1,57.0,34.0,4.3,0.0,91.0,70.0,POINT (10.74220 -5.59008)
362,47900,1147900,"Washington-Arlington-Alexandria, DC-VA-MD-WV",DC,3303526,6.0,2,"Washington Dulles, Washington National",IAD/DCA,172,56,228,0.00,6.00,6.00,NaN,NaN,N/A,1.60 -7.00%,2.70,9000.0,Plus 0.2% administrative funding assessment,8.95,6.0,Yes,No entity-level tax measured by net worth/capi...,8.25,Single factor sales,10.0,Yes,106.0,17.0,31.0,13588.0,12528.0,-0.078,NaN,6298330.0,6627605.0,1.02,38.0,103891.0,432787.0,0.528,75.5,Washington-Arlington-Alexandria DC-VA,159.0,115.7,271.4,113.2,105.5,92.8,121.6,3033.0,2708.0,9263.0,68430.0,11632.0,11486.0,6689.0,1956.0,583551.0,95276.0,74.0,110.0,104.0,79.0,87.0,141.0,3.0,7.0,42.0,24.0,3.9,0.0,86.0,66.0,POINT (14.66352 1.48537)
363,47940,1947940,"Waterloo-Cedar Falls, IA",IA,83334,4.7,0,None,None,0,0,0,0.94,6.00,6.94,1632.0,17.0,No,0.013,0.00,30600.0,26,8.53,9.0,Yes,No entity-level tax measured by net worth/capi...,12.00,Single factor sales,4.0,Yes,91.0,0.0,0.0,261.0,96.0,-0.632,67.0,170793.0,172797.0,0.23,38.0,56308.0,156553.0,0.298,33.1,Waterloo-Cedar Falls IA,87.4,93.3,78.6,86.2,101.4,102.1,86.9,725.0,147.0,972.0,2404.0,2601.0,74.0,0.0,0.0,22590.0,4390.0,6.0,87.0,90.0,106.0,96.0,57.0,0.9,7.6,26.0,8.0,4.8,0.0,83.0,62.0,POINT (3.04697 3.91216)
364,48060,3648060,"Watertown-Fort Drum, NY",NY,43035,7.1,0,None,None,0,0,0,4.52,4.00,8.52,2902.0,4.0,No,0.60 - 7.90%,3.20,11600.0,"*2019, Includes 0.075% Re-employment Service F...",8.82,8.0,Yes,Capital value tax is part of overall franchise...,6.50,Single factor sales,1.0,No,98.0,0.0,0.0,138.0,129.0,-0.065,18.0,117481.0,114797.0,-0.46,35.0,52804.0,159294.0,0.240,39.1,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4500.0,564.0,1.0,85.0,84.0,103.0,129.0,51.0,2.9,31.1,29.0,9.0,3.0,0.0,79.0,60.0,POINT (14.65636 6.81604)
365,48140,5548140,"Wausau, WI",WI,72490,3.5,0,None,None,0,0,0,0.46,5.00,5.46,1655.0,14.0,Yes,0.00- 12.00%,NaN,14000.0,Includes solvency surtax,7.65,4.0,Yes,No entity-level tax measured by net worth/capi...,7.90,Single factor sales,1.0,No,101.0,0.0,0.0,274.0,180.0,-0.343,NaN,164516.0,166003.0,0.18,42.0,61160.0,168073.0,0.248,22.4,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,15.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,89.0,98.0,88.0,108.0,65.0,1.0,13.5,23.0,6.0,3.8,0.0,80.0,59.0,POINT (4.87017 6.53120)
366,48260,3948260,"Weirton-Steubenville, WV-OH",WV,49796,7.9,0,None,None,0,0,0,0.41,6